In [7]:
import pandas as pd
import numpy as np

In [ ]:
train = pd.read_json(open("./data/train.json", "r"))

In [ ]:
any(pd.to_datetime(train["created"]).dt.year != 2016)

In [ ]:
print(train[['photos']])

In [ ]:
t = train.groupby('manager_id')['manager_id'].value_counts()

In [ ]:
s=train.groupby(['manager_id','interest_level']).size().unstack()

In [ ]:
import numpy as np

In [ ]:
from numba import vectorize, float32, int32
from numba.types import UniTuple

In [ ]:
@autojit
def intlevel_freqcount(a,b,c):
    a,b,c = np.nan_to_num([a,b,c])
    tot = sum([a,b,c])
    return pd.Series([a/tot,b/tot,c/tot,tot])
    

In [ ]:
intlevel_freqcount(1,2,5)

In [ ]:
s.apply(np.nan_to_num)

In [ ]:
s.reset_index().apply(np.nan_to_num).merge(t,how='left',left_on='manager_id',right_index=True)

In [ ]:
t

In [ ]:
    df_manager = (pd.concat([train['manager_id'], 
                           pd.get_dummies(train['interest_level'])], axis = 1)
                                        .groupby('manager_id')
                                        .mean()
                                        .rename(columns = lambda x: 'manager_percent_' + x)
                                           )


In [ ]:
df_manager

In [ ]:
t = train.groupby('manager_id')['manager_id'].value_counts()

In [ ]:


pd.concat([train['manager_id'],pd.get_dummies(train['interest_level'])], axis = 1).groupby('manager_id').mean()

In [ ]:
df_manager['test']=train.groupby('manager_id').size()

In [ ]:
df_manager

In [ ]:
df_mngr = (pd.concat([train['manager_id'], 
                           pd.get_dummies(train['interest_level'])], axis = 1)
                                        .groupby('manager_id')
                                        .transform('mean')
                                        .rename(columns = lambda x: 'mngr_percent_' + x)
                                           )



In [ ]:
df_mngr

In [ ]:
# get ixes for unranked managers...
unrkd_mngrs_ixes = df_mngr['manager_count']<20
# ... and ranked ones
rkd_mngrs_ixes = ~unrkd_mngrs_ixes

# compute mean values from ranked managers and assign them to unranked ones
mean_val = df_mngr.loc[rkd_mngrs_ixes,
                           ['mngr_percent_high',
                            'mngr_percent_low',
                            'mngr_percent_medium',
                            'manager_skill']].mean()
df_mngr.loc[unrkd_mngrs_ixes, ['mngr_percent_high',
                                    'mngr_percent_low',
                                    'mngr_percent_medium',
                                    'manager_skill']] = mean_val.values

In [ ]:
train['description'][100004]

In [ ]:
import matplotlib.pyplot as plt
from sklearn.cluster import DBSCAN
from geopy.distance import great_circle
from shapely.geometry import MultiPoint
from sklearn import metrics
from hdbscan import HDBSCAN
import time
%matplotlib inline

In [ ]:
# define the number of kilometers in one radian
kms_per_radian = 6371.0088

In [ ]:
coords = train[['latitude','longitude']]

In [ ]:
coords

In [ ]:
# define epsilon as 200m, converted to radians for use by haversine
epsilon = 0.2 / kms_per_radian

In [ ]:
start_time = time.time()
db = DBSCAN(eps=epsilon, min_samples=1, algorithm='ball_tree', metric='haversine', n_jobs=-1).fit(np.radians(coords))
cluster_labels = db.labels_

# get the number of clusters
num_clusters = len(set(cluster_labels))

# all done, print the outcome
message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
print(message.format(len(train), num_clusters, 100*(1 - float(num_clusters) / len(train)), time.time()-start_time))
#print('Silhouette coefficient: {:0.03f}'.format(metrics.silhouette_score(coords, cluster_labels)))

In [ ]:
clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
print(clusters)
centermost_points = clusters.map(_get_centermost_point)

In [ ]:
def _get_centermost_point(cluster):
        centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
        centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
        return tuple(centermost_point)
    
def _trans(df):
        
        coords = df[['latitude','longitude']].as_matrix()
        
        start_time = time.time()
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/' 
                   ).fit(coords)
        cluster_labels = db.labels_
        
        # get the number of clusters
        num_clusters = len(set(cluster_labels))
        message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
        print(message.format(len(df), num_clusters, 100*(1 - float(num_clusters) / len(df)), time.time()-start_time))
        
        # turn the clusters in to a pandas series, where each element is a cluster of points
        clusters = pd.Series([coords[cluster_labels==n] for n in range(num_clusters)])
        print(clusters)
        centermost_points = clusters.map(_get_centermost_point)
        
        #Retrieve the lists of centermost points
        new_lat, new_long = zip(*centermost_points)
        return df.assign(
            cluster_lat = new_lat,
            cluster_long = new_long
        )

In [ ]:
test = pd.read_json(open("./data/test.json", "r"))

In [ ]:
_trans(test)

In [ ]:
from hdbscan import HDBSCAN

In [ ]:
        coords = train[['latitude','longitude']]
        
        start_time = time.time()
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/'
                   ).fit(coords)
        cluster_labels = db.labels_
        
        # get the number of clusters
        num_clusters = cluster_labels.max()
        message = 'Clustered {:,} points down to {:,} clusters, for {:.1f}% compression in {:,.2f} seconds'
        print(message.format(len(train), num_clusters, 100*(1 - float(num_clusters) / len(train)), time.time()-start_time))
        
        # turn the clusters in to a pandas series, where each element is a cluster of points
        clusters = pd.Series([coords.iloc[cluster_labels==n] for n in range(num_clusters)])

In [ ]:
cluster_labels

In [ ]:
cluster_labels.max()

In [ ]:
coords_new = coords

In [ ]:
c = coords.assign( cluster = db.labels_)

In [ ]:
gb = c.groupby('cluster')['latitude','longitude']

In [ ]:
cluster_centers = gb.agg(lambda x: _get_centermost_point(x.values))

In [ ]:
def _get_centermost_point(cluster):
        centroid = (MultiPoint(cluster).centroid.x, MultiPoint(cluster).centroid.y)
        centermost_point = min(cluster, key=lambda point: great_circle(point, centroid).m)
        lat,long,label = centermost_point
        return lat,long

In [ ]:
c.merge(cluster_centers, left_on='cluster', right_index=True,how='left',suffixes=('', '_cluster'))

In [ ]:
cluster_centers

In [ ]:
coords_test = test[['latitude','longitude']]

In [ ]:
db.approximate_predict(coords_test)

In [ ]:
        db = HDBSCAN(min_samples=1,
                    metric='haversine',
                    algorithm='boruvka_balltree',
                    core_dist_n_jobs=-1,
                    memory='./__pycache__/',
                    prediction_data=True
                   )

In [ ]:
coords = train[['latitude','longitude']]

In [ ]:
train2 = train.assign( cluster = db.fit_predict(coords) )

In [ ]:
train2[['cluster','latitude','longitude']].groupby('cluster')['latitude','longitude'].agg(lambda x: _get_centermost_point(x.values))

In [ ]:
from hdbscan import approximate_predict

In [ ]:
a = approximate_predict(db,coords_test)

In [ ]:
a[0].shape

In [ ]:
a[1]==1

In [ ]:
import matplotlib.image as mpimg
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
img = mpimg.imread('./data/images/7211212/7211212_1ed4542ec81621d70d1061aa833e669c.jpg')

In [ ]:
imgplot = plt.imshow(img)

In [ ]:
train[train.display_address==""]

In [ ]:
import urllib, json

def ParsNYGeo(jBlob):
  try:
    jBlob['candidates']
  except KeyError:  
    data = [None,None,None]
  else:
    add = jBlob['candidates'][0]['address']
    y = jBlob['candidates'][0]['location']['y']
    x = jBlob['candidates'][0]['location']['x']
    data = [add,x,y]
  return data

def NYSGeo(Add, WKID=4326):
  base = "http://gisservices.dhses.ny.gov/arcgis/rest/services/Locators/SAM_composite/GeocodeServer/findAddressCandidates?SingleLine="
  wkid = "&maxLocations=1&outSR=4326"
  end = "&f=pjson"
  mid = Add.replace(' ','+')
  MyUrl = base + mid + wkid + end
  soup = urllib.request.urlopen(MyUrl)
  jsonRaw = soup.read()
  jsonData = json.loads(jsonRaw)
  MyDat = ParsNYGeo(jsonData)
  return MyDat

t1 = "100 Washington Ave, Albany, NY"
t2 = "100 Washington Ave, Poop"

Out = NYSGeo(t1)
print(Out)

Empt = NYSGeo(t2)
print(Empt)

In [ ]:
metadata = pd.read_json(open("./data_preprocessed/metadata.json", "r"))

In [ ]:
metadata = pd.read_csv("./data_preprocessed/metadata.csv", "r", encoding='latin_1')

In [38]:
import paratext

In [ ]:
dict_frame, levels = paratext.load_csv_to_dict("./data_preprocessed/metadata.csv")

In [ ]:
dict_frame

In [ ]:
  for key in dict_frame.keys():
      print(key, repr(dict_frame[key][0:5]), levels.get(key, None))

In [ ]:
df = pd.read_csv

In [5]:
%%bash
xsv headers './data_preprocessed/metadata.csv'

1     SourceFile
2     About
3     AccelerationVector
4     Accentuation
5     AccessorySerialNumber
6     AccessoryType
7     ActiveD-Lighting
8     AddOriginalDecisionData
9     AdvancedSceneMode
10    AdvancedSceneType
11    AEBAutoCancel
12    AEBBracketValue
13    AEBSequence
14    AEBSequenceAutoCancel
15    AELock
16    AELockButton
17    AESetting
18    AF-CPrioritySelection
19    AF-OnForMB-D11
20    AF-SPrioritySelection
21    AFAperture
22    AFAreaHeight
23    AFAreaHeights
24    AFAreaIllumination
25    AFAreaMode
26    AFAreaModeSetting
27    AFAreas
28    AFAreaWidth
29    AFAreaWidths
30    AFAreaXPosition
31    AFAreaXPositions
32    AFAreaYPosition
33    AFAreaYPositions
34    AFAssist
35    AFAssistBeam
36    AFAssistLamp
37    AFDuringLiveView
38    AFFineTune
39    AFFineTuneAdj
40    AFFineTuneIndex
41    AFIlluminator
42    AFImageHeight
43    AFImageWidth
44    AFInfo2Version
45    AFMicroAdjMode
46    AFMicroadjustment
47    AFMicroAdjValue
48    AFMode
49    A

In [8]:
stats = pd.read_csv('./data_preprocessed/metadata_stats.csv')

In [9]:
stats

,field,type,sum,min,max,min_length,max_length,mean,stddev
0,SourceFile,Unicode,NaN,../data/images/6811957/6811957_33d08c8dc440c89...,../data/images/7761779/6991915_fec706c6957a554...,67,75,NaN,NaN
1,About,Unicode,NaN,uuid:03965c1d-f45d-11dc-a61f-f79a93750b2f,uuid:ff7009d6-ec8d-11dc-88b1-ee2a3cf8db56,0,41,NaN,NaN
2,AccelerationVector,Unicode,NaN,-0.0001949409221 -1.000346861 -0.1218700246,1.030815258 0.006586884925 -0.008118074326,0,45,NaN,NaN
3,Accentuation,Float,0.00,0,0,0,3,0.000000,0.000000
4,AccessorySerialNumber,Integer,0.00,0,0,0,7,NaN,NaN
5,AccessoryType,Unicode,NaN,NO-ACCESSORY,NO-ACCESSORY,0,12,NaN,NaN
6,ActiveD-Lighting,Unicode,NaN,Auto,Off,0,4,NaN,NaN
7,AddOriginalDecisionData,Unicode,NaN,Off,Off,0,3,NaN,NaN
8,AdvancedSceneMode,Unicode,NaN,Beach,Unknown (60 7),0,35,NaN,NaN
9,AdvancedSceneType,Integer,2815.00,1,7,0,1,2.704131,2.066352


In [43]:
freq = pd.read_csv('./data_preprocessed/metadata_frequency.csv', encoding="ISO-8859-1")

In [49]:
freq[~(freq.value == '(NULL)')].groupby('field')['field','count'].sum().to_csv('metadata_count.csv')

In [51]:
freq[(freq.value == '(NULL)')].to_csv('metadata_nullcount.csv')

In [54]:
count2 = pd.read_csv('./data_preprocessed/metadata_count.csv')

In [60]:
count2[count2['count']>=10000].to_csv('metadata_count10000.csv')

In [62]:
metadata = pd.read_csv('./data_preprocessed/metadata_subset.csv')

/usr/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,23,32,61) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [65]:
metadata

,BitsPerSample,BlueMatrixColumn,BlueTRC,CMMFlags,CircleOfConfusion,ColorComponents,ColorSpace,ColorSpaceData,ComponentsConfiguration,Compression,ConnectionSpaceIlluminant,CustomRendered,DeviceAttributes,DeviceManufacturer,DeviceMfgDesc,DeviceModel,DeviceModelDesc,EncodingProcess,ExifByteOrder,ExifImageHeight,ExifImageWidth,ExifToolVersion,ExifVersion,ExposureCompensation,ExposureMode,ExposureProgram,FileInodeChangeDate,FilePermissions,FileSize,FileType,FileTypeExtension,Flash,FlashpixVersion,GreenMatrixColumn,GreenTRC,ImageHeight,ImageSize,ImageWidth,JFIFVersion,Luminance,MIMEType,Make,MeasurementBacking,MeasurementFlare,MeasurementGeometry,MeasurementIlluminant,MeasurementObserver,MediaBlackPoint,MediaWhitePoint,Megapixels,MeteringMode,Orientation,PrimaryPlatform,ProfileCMMType,ProfileClass,ProfileConnectionSpace,ProfileCopyright,ProfileCreator,ProfileDateTime,ProfileDescription,ProfileFileSignature,ProfileID,ProfileVersion,RedMatrixColumn,RedTRC,RenderingIntent,ResolutionUnit,ScaleFactor35efl,SceneCaptureType,Technology,ViewingCondDesc,ViewingCondIlluminant,ViewingCondIlluminantType,ViewingCondSurround,WhiteBalance,XResolution,YCbCrPositioning,YCbCrSubSampling,YResolution
0,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,186 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
1,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,191 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
2,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,147 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
3,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,212 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
4,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,158 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
5,8,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:12+00:00,r-xr-xr-x,145 kB,JPEG,jpg,NaN,NaN,NaN,NaN,480,640x480,640,1.01,NaN,image/jpeg,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.307,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,inches,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,72.0,NaN,YCbCr4:2:0 (2 2),72.0
6,8,0.14307 0.06061 0.7141,"(Binary data 2060 bytes, use -b option to extr...","Not Embedded, Independent",NaN,3,NaN,RGB,NaN,NaN,0.9642 1 0.82491,NaN,"Reflective, Glossy, Positive, Color",IEC,IEC http://www.iec.ch,sRGB,IEC 61966-2.1 Default RGB colour space - sRGB,"Baseline DCT, Huffman coding",NaN,NaN,NaN,10.4,NaN,NaN,NaN,NaN,2017:02:26 21:14:11+00:00,r-xr-xr-x,172 kB

In [64]:
pd.options.display.max_columns = 80